In [8]:
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np


In [2]:
def load(f):
    # Move up one directory when loading the data
    file_path = os.path.join('../../../', f)
    return np.load(file_path)['arr_0']

# Load the data
X_train = load('kmnist-train-imgs.npz').reshape(-1, 28*28) / 255.0
x_test = load('kmnist-test-imgs.npz').reshape(-1, 28*28) / 255.0
y_train = load('kmnist-train-labels.npz')
y_test = load('kmnist-test-labels.npz')

In [3]:
# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from models.MLP import create_mlp

# Define the input shape and number of classes
input_shape = X_train.shape[1]  # 784 for KMNIST
num_classes = y_train.max() + 1


In [7]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define a range of hyperparameters to test
# 1, 2, or 3 layers with 2000 neurons total
architectures = [
    {0: 2000},
    {0: 1000, 1: 1000},
    {0: 500, 1: 1000, 2: 500},
]
# Dropout of 0.0 or 0.5
dropout_rates = [0.0, 0.5]
# Optimizer of adam only
optimizers = ['adam']
# Learning rate of 0.001 or 0.01
learning_rates = [0.001, 0.01]
# Activation of relu or tanh
activations = ['relu', 'tanh']

# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
# Store results
results = []
best_model = None
# Test different combinations
for arch in architectures:
    for dropout_rate in dropout_rates:
        for optimizer in optimizers:
                for lr in learning_rates:
                    for activation in activations:
                        print(f"Testing: Architecture: {arch}, Dropout: {dropout_rate}, Optimizer: {optimizer}, Learning Rate: {lr}, Activation: {activation}")
                        # Create the MLP model
                        model = create_mlp(input_shape, num_classes, arch, activation=activation, learning_rate=lr, optimizer=optimizer, dropout_rate=dropout_rate)
                        
                        # Define callback for early stopping
                        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
                        
                        # Train the model
                        history = model.fit(
                            x_train, y_train,
                            batch_size=64,
                            epochs=50,
                            validation_data=(x_val, y_val),
                            callbacks=[early_stopping],
                            verbose=1
                        )
                        # Save the best model based on validation accuracy
                        if not best_model or history.history['val_accuracy'][-1] > best_model['val_accuracy']:
                            best_model = {
                                'architecture': arch,
                                'dropout_rate': dropout_rate,
                                'optimizer': optimizer,
                                'learning_rate': lr,
                                'activation': activation,
                                'val_accuracy': history.history['val_accuracy'][-1],
                                'val_loss': history.history['val_loss'][-1],
                                'epochs_trained': len(history.history['val_accuracy'])
                            }
                            model.save('best_model.h5')
                        # Save the results
                        results.append({
                            'architecture': arch,
                            'dropout_rate': dropout_rate,
                            'optimizer': optimizer,
                            'learning_rate': lr,
                            'activation': activation,
                            'val_accuracy': max(history.history['val_accuracy']),
                            'val_loss': min(history.history['val_loss']),
                            'epochs_trained': len(history.history['val_loss'])
                        })

# Find the best model
best_model = min(results, key=lambda x: x['val_loss'])

print("\nBest Model Configuration:")
print(f"Architecture: {best_model['architecture']}")
print(f"Dropout Rate: {best_model['dropout_rate']}")
print(f"Optimizer: {best_model['optimizer']}")
print(f"Learning Rate: {best_model['learning_rate']}")
print(f"Activation: {best_model['activation']}")
print(f"Best Validation Accuracy: {best_model['val_accuracy']:.4f}")
print(f"Best Validation Loss: {best_model['val_loss']:.4f}")
print(f"Epochs Trained: {best_model['epochs_trained']}")

# Load the best model
best_model = tf.keras.models.load_model('best_model.h5')

# Evaluate the best model on the test set
test_loss, test_accuracy = best_model.evaluate(x_test, y_test, verbose=0)
print(f"\nBest Model Test Accuracy: {test_accuracy:.4f}")
print(f"Best Model Test Loss: {test_loss:.4f}")

Testing: Architecture: {0: 2000}, Dropout: 0.0, Optimizer: adam, Learning Rate: 0.001, Activation: relu
Epoch 1/50
616/616 [==============================] - 2s 3ms/step - loss: 0.3517 - accuracy: 0.8909 - val_loss: 0.2046 - val_accuracy: 0.9364
Epoch 2/50
616/616 [==============================] - 1s 2ms/step - loss: 0.1342 - accuracy: 0.9608 - val_loss: 0.1435 - val_accuracy: 0.9568
Epoch 3/50
616/616 [==============================] - 1s 2ms/step - loss: 0.0700 - accuracy: 0.9786 - val_loss: 0.1641 - val_accuracy: 0.9522
Epoch 4/50
616/616 [==============================] - 2s 2ms/step - loss: 0.0370 - accuracy: 0.9895 - val_loss: 0.1348 - val_accuracy: 0.9614
Epoch 5/50
616/616 [==============================] - 2s 3ms/step - loss: 0.0207 - accuracy: 0.9945 - val_loss: 0.1466 - val_accuracy: 0.9616
Epoch 6/50
616/616 [==============================] - 2s 3ms/step - loss: 0.0222 - accuracy: 0.9932 - val_loss: 0.1387 - val_accuracy: 0.9584
Epoch 7/50
616/616 [========================

In [15]:
# Load the best model
best_model = tf.keras.models.load_model('best_model.h5')

# Get predictions on the test set
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

print("\nModel Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# per-class metrics
print("\nPer-class Metrics:")
class_precision = precision_score(y_test, y_pred_classes, average=None)
class_recall = recall_score(y_test, y_pred_classes, average=None)
class_f1 = f1_score(y_test, y_pred_classes, average=None)

for i in range(num_classes):
    print(f"Class {i}:")
    print(f"  Precision: {class_precision[i]:.4f}")
    print(f"  Recall: {class_recall[i]:.4f}")
    print(f"  F1-Score: {class_f1[i]:.4f}")

313/313 [==============================] - 0s 887us/step

Model Performance Metrics:
Accuracy: 0.9140
Precision: 0.9148
Recall: 0.9140
F1-Score: 0.9140

Per-class Metrics:
Class 0:
  Precision: 0.9304
  Recall: 0.9360
  F1-Score: 0.9332
Class 1:
  Precision: 0.9184
  Recall: 0.8890
  F1-Score: 0.9035
Class 2:
  Precision: 0.8628
  Recall: 0.8740
  F1-Score: 0.8684
Class 3:
  Precision: 0.9033
  Recall: 0.9620
  F1-Score: 0.9317
Class 4:
  Precision: 0.9145
  Recall: 0.8880
  F1-Score: 0.9011
Class 5:
  Precision: 0.9583
  Recall: 0.8970
  F1-Score: 0.9267
Class 6:
  Precision: 0.8821
  Recall: 0.9430
  F1-Score: 0.9116
Class 7:
  Precision: 0.9432
  Recall: 0.9130
  F1-Score: 0.9278
Class 8:
  Precision: 0.8989
  Recall: 0.9250
  F1-Score: 0.9118
Class 9:
  Precision: 0.9364
  Recall: 0.9130
  F1-Score: 0.9246
